In [5]:
import numpy as np
import sklearn 
import math
import os
import matplotlib
import matplotlib.pyplot as plt
from scipy import signal


'''加载文件'''
def loadData(filePath):
    file = open(filePath,'rb')
    data = file.read()
    file.close()
    return data

'''得到所有文件名'''
def getFileList(rootPath):
    fileList_temp = []
    for root, dirs, files in os.walk(rootPath):
        fileList_temp.append(files)
    fileList = []
    for filter in fileList_temp[0]:
        if filter.split('.')[1] == 'txt':
            fileList.append(rootPath+'/'+filter)
    return fileList

'''把得到的加速度,角速度,标签，写入到文件中'''
def write2file(X,Y,X_path,Y_path):
    np.savetxt(X_path,X,delimiter=',')
    np.savetxt(Y_path,Y,delimiter=',')
    
    
'''从文件中获取加速度和角速度'''
def getAandG(data):
    data_len = len(data)
    a = [[] for i in range(3)]  # 加速度
    w = [[] for i in range(3)]  # 角速度
    index = 0
    while index < data_len:  
        if data[index] == 0x55:  # 包头
#             print("包头")
            if index + 7 < data_len:
                temp_a = data[index+3]
                temp_b = data[index+5]
                temp_c = data[index+7]
                if data[index+3] > 127:  # 说明是负数
                    temp_a = temp_a - 256
                if data[index+5] >127:
                    temp_b = temp_b - 256
                if data[index+7] >127:
                    temp_c = temp_c - 256
                if data[index+1] == 81: # 加速度输出
                    a[0].append(temp_a*256+data[index+2]) # x轴加速度
                    a[1].append(temp_b*256+data[index+4]) # y轴加速度
                    a[2].append(temp_c*256+data[index+6]) # z轴加速度
#                     print("a[x]=%d,a[y]=%d,a[z]=%d"%(a[0][-1],a[1][-1],a[2][-1]))
                    index += 11
                    continue
                elif data[index+1] == 82: # 角速度输出
                    w[0].append(temp_a*256+data[index+2]) # x轴角速度
                    w[1].append(temp_b*256+data[index+4]) # y轴角速度
                    w[2].append(temp_c*256+data[index+6]) # z轴角速度
#                     print("w[x]=%d,w[y]=%d,w[z]=%d"%(w[0][-1],w[1][-1],w[2][-1]))
                    index += 11
                    continue
                elif data[index+1] == 83: # 角度输出
                    index += 11
                else:  # 或者数据有丢失
                    index += 1
            else:  # 没有完整的数据了
                break
        else:  # 数据有丢失
            index += 1
    a = np.array(a)
    w = np.array(w)
    a = a / 32768 * 16    #单位为g
    w = w / 32768 * 2000  # 单位为°/s
    #print(a.shape+w.shape)
    return a,w

'''获取指定目录下的原始数据，将转换后的数据合并成一个张量
   rootPath为目标路径，start-起始文件，end-截至文件
'''
def convertData(fileList,start=-1,end=-1):
    acc = np.empty((3,0))#创建一个空数组，用于存放加速度的集合
    rad = np.empty((3,0))
    if(end>start&start>0):
        targetFiles = fileList[start:end]
    else:
        targetFiles=fileList
        
    for filePath in targetFiles:
        #filePath = rootPath+'/'+file
        data = loadData(filePath)
        a,w = getAandG(data)
        len_diff = len(a[1])-len(w[1])
        if len_diff !=0:#数据对齐
            if len_diff>0:#加速度数据长
                w = np.hstack((w,np.zeros((3,len_diff))))
            elif len_diff<0:#角速度数据长
                a = np.hstack((a,np.zeros((3,-len_diff))))   
#         print(len(a[1])==len(w[1]))#判断是否对齐
# 将数据前后100各采样点换成0 ，消除上电断电对数据的影响
        a,w = cutSEpoint(a,w)
        #print(acc.shape+rad.shape)
        acc = np.hstack((acc,a))
        rad = np.hstack((rad,w))
    print(acc.shape+rad.shape)
    return acc,rad

'''将数据前后100各采样点换成0 ，消除上电断电对数据的影响'''
def cutSEpoint(acc,rad,lenth=100):
    for i in range(lenth):
        acc[0][i]=0
        acc[1][i]=0
        acc[2][i]=0
        rad[0][i]=0
        rad[1][i]=0
        rad[2][i]=0
        acc[0][-i]=0
        acc[1][-i]=0
        acc[2][-i]=0
        rad[0][-i]=0
        rad[1][-i]=0
        rad[2][-i]=0
    return acc,rad

'''三轴数据画图'''
def showPic(data):
    plt.figure(figsize=(20,15))
    plt.plot(range(0,len(data[0])),data[0],'r',range(0,len(data[1])),data[1],'g',range(0,len(data[2])),data[2],'b')
    label=["X","Y","Z"]
    plt.legend(label,loc=0,ncol=2)
#     plt.savefig("三轴数据滤波&切割后波形",dpi=600)
    plt.show()

'''过低通滤波'''
def smoothing(data,n=3,wn=0.02):
    b,a = signal.butter(n,wn,'low')  
    sf = signal.filtfilt(b,a,data)
    return sf

'''分割算法——acc,rad为待分隔数据，windowlen为窗口大小，strides为单步滑动大小，
threshold1为用于定义切断信号状态的阈值,threshold2用于定义窗口内有多少个1时判断为1'''
def getSegFun(acc,rad,windowlen=100,strides=100,threshold1=6,threshold2=50):
    #用于保存各动作起点终点位置
    start_point =[]
    end_point = []
    pointSignal = np.arange(len(rad[1]))#声明一个切断信号,因为六轴数据已经对齐，数据长度相同，所以统一用rad[1]的长度代表整体长度
    for i in range(len(rad[1])):
        #tmp= math.sqrt(abs(rad[0][i])**2+abs(rad[1][i])**2+abs(rad[2][i])**2)#只是其中一种方法
        tmp= abs(rad[0][i])+abs(rad[1][i])+abs(rad[2][i])#只是其中一种方法
        if(tmp>threshold1):
            pointSignal[i]=1
        else:
            pointSignal[i]=0
    # print(pointSignal)
    state = 0
    window = np.arange(windowlen)
    for i in range(0,len(pointSignal),strides):
        if(i+strides<len(pointSignal)):
            window = pointSignal[i:i+strides]
            if(np.sum(window)>threshold2):
                pointSignal[i:i+strides]=1
                if(state==0):#状态转换时，留出一小段保留数据。
                    pointSignal[i-100:i]=1
                state = 1
            else:
                pointSignal[i:i+strides]=0
                if(state==1):#状态转换时，留出一小段保留数据。
                    pointSignal[i:i+100]=1
                state = 0
        else:
            window = pointSignal[i:]
            if(np.sum(window)>threshold2):
                pointSignal[i:]=1
            else:
                pointSignal[i:]=0
    #acc1 = acc*pointSignal
    #rad1 = rad*pointSignal
    #return acc1,rad1
    return pointSignal

'''获取切割端点'''
def getEndPoint(pointSignal):  
    # 用于保存各动作起点终点位置
    start_point =[]
    end_point = []
    # 状态位
    state = 0
    for i in range(len(pointSignal)):
        if(state == 0 and pointSignal[i] == 1):
            start_point.append(i)
            state = 1
        elif(state == 1 and pointSignal[i] == 0):
            end_point.append(i)
            state = 0
#    print("切割段数：%d"%len(end_point))
    #print(len(start_point))
    return start_point,end_point

In [6]:
#加载数据
fileList = getFileList("data/11.6/X")
fileList +=getFileList("data/11.15/X")
fileList +=getFileList("data/11.25/X")
fileList +=getFileList("data/12.3/X")
print("样本个数：%d"%len(fileList))
#showPic(acc)
#np.loadtxt("E:/Car data/data/11.25/Y/label.txt").astype(np.int).reshape(-1,1)

样本个数：107


In [7]:
#数据预处理
acc,rad = convertData(fileList)
'''1.过低通滤波器'''
s_acc = smoothing(acc,n=3,wn=0.02)
s_rad = smoothing(rad,n=3,wn=0.02)
'''2.根据平滑后的数据进行切断，并获取端点'''
#pointSignal = getSegFun(s_acc,s_rad,windowlen=100,strides=100,threshold1=3,threshold2=57)
windowlen = []
threshold1 = []
threshold2 = []
max_lenth =[]
pointSignal=[]
for w in range(50,100):
    for t1 in range(2,10):
        for t2 in range(10,100): 
            p = getSegFun(s_acc,s_rad,windowlen=w,strides=100,threshold1=t1,threshold2=t2)            
            start_point,end_point = getEndPoint(p)
            if(len(start_point)==107): 
                print("window:%d  t1:%d  t2:%d"%(w,t1,t2))
                '''3.切分数据'''
                #输出最大长度
                lenth = 0 
                for i in range(len(end_point)):
                    if(lenth < end_point[i]- start_point[i]):
                        lenth = end_point[i]- start_point[i]
                if(lenth<3000):
                    print("window:%d  t1:%d  t2:%d  最大长度：%d"%(w,t1,t2,lenth))
                    max_lenth.append(lenth)
                    windowlen.append(w)
                    threshold1.append(t1)
                    threshold2.append(t2)
                    pointSignal.append(p)

#画出切断函数
# plt.figure(figsize=(20,15))
# plt.plot(range(0,len(pointSignal)),pointSignal)
# plt.show()

(3, 182158, 3, 182158)
window:50  t1:2  t2:15
window:50  t1:3  t2:76
window:50  t1:3  t2:77
window:50  t1:3  t2:78
window:50  t1:5  t2:42
window:50  t1:7  t2:14
window:50  t1:7  t2:15
window:50  t1:7  t2:16
window:51  t1:2  t2:15
window:51  t1:3  t2:76
window:51  t1:3  t2:77
window:51  t1:3  t2:78
window:51  t1:5  t2:42
window:51  t1:7  t2:14
window:51  t1:7  t2:15
window:51  t1:7  t2:16
window:52  t1:2  t2:15
window:52  t1:3  t2:76
window:52  t1:3  t2:77
window:52  t1:3  t2:78
window:52  t1:5  t2:42
window:52  t1:7  t2:14
window:52  t1:7  t2:15
window:52  t1:7  t2:16
window:53  t1:2  t2:15
window:53  t1:3  t2:76
window:53  t1:3  t2:77
window:53  t1:3  t2:78
window:53  t1:5  t2:42
window:53  t1:7  t2:14
window:53  t1:7  t2:15
window:53  t1:7  t2:16
window:54  t1:2  t2:15
window:54  t1:3  t2:76
window:54  t1:3  t2:77
window:54  t1:3  t2:78
window:54  t1:5  t2:42
window:54  t1:7  t2:14
window:54  t1:7  t2:15
window:54  t1:7  t2:16
window:55  t1:2  t2:15
window:55  t1:3  t2:76
window:55  

window:94  t1:5  t2:42
window:94  t1:7  t2:14
window:94  t1:7  t2:15
window:94  t1:7  t2:16
window:95  t1:2  t2:15
window:95  t1:3  t2:76
window:95  t1:3  t2:77
window:95  t1:3  t2:78
window:95  t1:5  t2:42
window:95  t1:7  t2:14
window:95  t1:7  t2:15
window:95  t1:7  t2:16
window:96  t1:2  t2:15
window:96  t1:3  t2:76
window:96  t1:3  t2:77
window:96  t1:3  t2:78
window:96  t1:5  t2:42
window:96  t1:7  t2:14
window:96  t1:7  t2:15
window:96  t1:7  t2:16
window:97  t1:2  t2:15
window:97  t1:3  t2:76
window:97  t1:3  t2:77
window:97  t1:3  t2:78
window:97  t1:5  t2:42
window:97  t1:7  t2:14
window:97  t1:7  t2:15
window:97  t1:7  t2:16
window:98  t1:2  t2:15
window:98  t1:3  t2:76
window:98  t1:3  t2:77
window:98  t1:3  t2:78
window:98  t1:5  t2:42
window:98  t1:7  t2:14
window:98  t1:7  t2:15
window:98  t1:7  t2:16
window:99  t1:2  t2:15
window:99  t1:3  t2:76
window:99  t1:3  t2:77
window:99  t1:3  t2:78
window:99  t1:5  t2:42
window:99  t1:7  t2:14
window:99  t1:7  t2:15
window:99  

In [ ]:
file = open('max_lenth.txt','w+')
file.write(str(max_lenth))
file.close()
file = open('windowlen.txt','w+')
file.write(str(windowlen))
file.close()
file = open('threshold1.txt','w+')
file.write(str(threshold1))
file.close()
file = open('threshold2.txt','w+')
file.write(str(threshold2))
file.close()
file = open('pointSignal.txt','w+')
file.write(str(pointSignal))
file.close()

In [16]:
pointSignal = []
for w in windowlen:
    for t1 in threshold1:
        for t2 in threshold2: 
            p = getSegFun(s_acc,s_rad,windowlen=w,strides=100,threshold1=t1,threshold2=t2)
            pointSignal.append(p)
            start_point,end_point = getEndPoint(p)
            if(len(start_point)==107): 
                #print("window:%d  t1:%d  t2:%d"%(w,t1,t2))
                '''3.切分数据'''
                #输出最大长度
                lenth = 0 
                for i in range(len(end_point)):
                   #if(lenth < end_point[i]- start_point[i]):
                    lenth = end_point[i]- start_point[i]
                    print(lenth)
                max_lenth.append(lenth)
                print("window:%d  t1:%d  t2:%d  最大长度：%d "%(w,t1,t2,lenth))
       #         windowlen.append(w)
         #       threshold1.append(t1)
        #        threshold2.append(t2)

400
1300
500
500
1500
1500
400
700
1700
400
400
800
800
2200
700
600
300
400
300
800
700
800
300
300
300
500
500
300
2200
2200
700
1900
500
1000
2100
600
300
1400
300
700
2000
800
800
700
2100
300
300
2300
300
800
2300
400
800
400
300
2400
2400
500
300
300
300
2300
2600
2300
2400
300
300
2100
1000
600
300
800
2500
300
700
600
300
400
300
400
700
600
700
2400
900
900
2200
2400
300
300
400
300
300
3000
800
2500
800
800
2300
2500
2200
700
2800
700
900
800
300
window:50  t1:2  t2:84  最大长度：300 
400
1300
500
500
1400
1500
400
700
1700
400
400
800
800
2200
700
600
300
400
300
800
700
800
300
300
300
500
500
300
2200
2200
700
1900
500
1000
2100
600
300
1400
300
700
2000
800
800
700
2100
300
300
2300
300
800
2300
400
800
400
300
2400
2400
500
300
300
300
2300
2600
2300
2400
300
300
2100
1000
600
300
800
2500
300
700
600
300
300
300
400
700
600
700
2400
900
900
2200
2400
300
300
300
300
300
3000
800
2500
800
700
2300
2500
2200
700
2800
700
900
800
300
window:50  t1:2  t2:85  最大长度：300 
400
1300
5

300
400
300
800
700
800
300
300
300
500
500
300
2200
2200
700
1900
500
1000
2100
600
300
1400
300
700
2000
800
800
700
2100
300
300
2300
300
800
2300
400
800
400
300
2400
2400
500
300
300
300
2300
2600
2300
2400
300
300
2100
1000
600
300
800
2500
300
700
600
300
300
300
400
700
600
700
2400
900
900
2200
2400
300
300
300
300
300
3000
800
2500
800
700
2300
2500
2200
700
2800
700
900
800
300
window:50  t1:2  t2:85  最大长度：300 
400
1300
500
500
1500
1500
400
700
1700
400
400
800
800
2200
700
600
300
400
300
800
700
800
300
300
300
500
500
300
2200
2200
700
1900
500
1000
2100
600
300
1400
300
700
2000
800
800
700
2100
300
300
2300
300
800
2300
400
800
400
300
2400
2400
500
300
300
300
2300
2600
2300
2400
300
300
2100
1000
600
300
800
2500
300
700
600
300
400
300
400
700
600
700
2400
900
900
2200
2400
300
300
400
300
300
3000
800
2500
800
800
2300
2500
2200
700
2800
700
900
800
300
window:50  t1:2  t2:84  最大长度：300 
400
1300
500
500
1400
1500
400
700
1700
400
400
800
800
2200
700
600
300
400
30

400
1300
500
500
1500
1500
400
700
1700
400
400
800
800
2200
700
600
300
400
300
800
700
800
300
300
300
500
500
300
2200
2200
700
1900
500
1000
2100
600
300
1400
300
700
2000
800
800
700
2100
300
300
2300
300
800
2300
400
800
400
300
2400
2400
500
300
300
300
2300
2600
2300
2400
300
300
2100
1000
600
300
800
2500
300
700
600
300
400
300
400
700
600
700
2400
900
900
2200
2400
300
300
400
300
300
3000
800
2500
800
800
2300
2500
2200
700
2800
700
900
800
300
window:50  t1:2  t2:84  最大长度：300 
400
1300
500
500
1400
1500
400
700
1700
400
400
800
800
2200
700
600
300
400
300
800
700
800
300
300
300
500
500
300
2200
2200
700
1900
500
1000
2100
600
300
1400
300
700
2000
800
800
700
2100
300
300
2300
300
800
2300
400
800
400
300
2400
2400
500
300
300
300
2300
2600
2300
2400
300
300
2100
1000
600
300
800
2500
300
700
600
300
300
300
400
700
600
700
2400
900
900
2200
2400
300
300
300
300
300
3000
800
2500
800
700
2300
2500
2200
700
2800
700
900
800
300
window:50  t1:2  t2:85  最大长度：300 
400
1300
5

800
700
2100
300
300
2300
300
800
2300
400
800
400
300
2400
2400
500
300
300
300
2300
2600
2300
2400
300
300
2100
1000
600
300
800
2500
300
700
600
300
300
300
400
700
600
700
2400
900
900
2200
2400
300
300
300
300
300
3000
800
2500
800
700
2300
2500
2200
700
2800
700
900
800
300
window:50  t1:2  t2:85  最大长度：300 
400
1300
500
500
1500
1500
400
700
1700
400
400
800
800
2200
700
600
300
400
300
800
700
800
300
300
300
500
500
300
2200
2200
700
1900
500
1000
2100
600
300
1400
300
700
2000
800
800
700
2100
300
300
2300
300
800
2300
400
800
400
300
2400
2400
500
300
300
300
2300
2600
2300
2400
300
300
2100
1000
600
300
800
2500
300
700
600
300
400
300
400
700
600
700
2400
900
900
2200
2400
300
300
400
300
300
3000
800
2500
800
800
2300
2500
2200
700
2800
700
900
800
300
window:50  t1:2  t2:84  最大长度：300 
400
1300
500
500
1400
1500
400
700
1700
400
400
800
800
2200
700
600
300
400
300
800
700
800
300
300
300
500
500
300
2200
2200
700
1900
500
1000
2100
600
300
1400
300
700
2000
800
800
700
21

800
800
2200
700
600
300
400
300
800
700
800
300
300
300
500
500
300
2200
2200
700
1900
500
1000
2100
600
300
1400
300
700
2000
800
800
700
2100
300
300
2300
300
800
2300
400
800
400
300
2400
2400
500
300
300
300
2300
2600
2300
2400
300
300
2100
1000
600
300
800
2500
300
700
600
300
400
300
400
700
600
700
2400
900
900
2200
2400
300
300
400
300
300
3000
800
2500
800
800
2300
2500
2200
700
2800
700
900
800
300
window:50  t1:2  t2:84  最大长度：300 
400
1300
500
500
1400
1500
400
700
1700
400
400
800
800
2200
700
600
300
400
300
800
700
800
300
300
300
500
500
300
2200
2200
700
1900
500
1000
2100
600
300
1400
300
700
2000
800
800
700
2100
300
300
2300
300
800
2300
400
800
400
300
2400
2400
500
300
300
300
2300
2600
2300
2400
300
300
2100
1000
600
300
800
2500
300
700
600
300
300
300
400
700
600
700
2400
900
900
2200
2400
300
300
300
300
300
3000
800
2500
800
700
2300
2500
2200
700
2800
700
900
800
300
window:50  t1:2  t2:85  最大长度：300 
400
1300
500
500
1500
1500
400
700
1700
400
400
800
800
22

...

700
2500
2400
300
500
300
400
300
2500
2600
2300
2500
300
300
300
2100
600
600
300
700
2300
800
300
400
300
400
700
700
700
2500
300
900
900
2400
2400
300
300
300
300
300
3000
300
600
2500
700
800
2200
2600
2300
700
2800
700
700
700
300
window:50  t1:4  t2:42  最大长度：300 
400
1400
500
1500
1500
500
700
1800
500
400
800
400
300
2200
700
300
500
500
300
400
800
800
400
300
500
300
300
2200
300
2100
800
2000
500
900
2100
300
300
1500
700
2100
300
500
800
600
2100
300
300
2300
300
900
2200
500
700
2500
2400
300
500
300
400
300
2500
2600
2300
2500
300
300
300
2100
600
600
300
700
2300
800
300
400
300
400
700
700
700
2500
300
900
900
2400
2400
300
300
300
300
300
3000
300
600
2500
700
800
2200
2600
2300
700
2800
700
700
700
300
window:50  t1:4  t2:42  最大长度：300 
400
1400
500
1500
1500
500
700
1800
500
400
800
400
300
2200
700
300
500
500
300
400
800
800
400
300
500
300
300
2200
300
2100
800
2000
500
900
2100
300
300
1500
700
2100
300
500
800
600
2100
300
300
2300
300
900
2200
500
700
2500
2400


700
300
500
500
300
400
800
800
400
300
500
300
300
2200
300
2100
800
2000
500
900
2100
300
300
1500
700
2100
300
500
800
600
2100
300
300
2300
300
900
2200
500
700
2500
2400
300
500
300
400
300
2500
2600
2300
2500
300
300
300
2100
600
600
300
700
2300
800
300
400
300
400
700
700
700
2500
300
900
900
2400
2400
300
300
300
300
300
3000
300
600
2500
700
800
2200
2600
2300
700
2800
700
700
700
300
window:50  t1:4  t2:42  最大长度：300 
400
1400
500
1500
1500
500
700
1800
500
400
800
400
300
2200
700
300
500
500
300
400
800
800
400
300
500
300
300
2200
300
2100
800
2000
500
900
2100
300
300
1500
700
2100
300
500
800
600
2100
300
300
2300
300
900
2200
500
700
2500
2400
300
500
300
400
300
2500
2600
2300
2500
300
300
300
2100
600
600
300
700
2300
800
300
400
300
400
700
700
700
2500
300
900
900
2400
2400
300
300
300
300
300
3000
300
600
2500
700
800
2200
2600
2300
700
2800
700
700
700
300
window:50  t1:4  t2:42  最大长度：300 
400
1400
500
1500
1500
500
700
1800
500
400
800
400
300
2200
700
300
500
50

900
300
400
300
2200
500
300
600
500
400
800
800
400
300
700
300
300
2400
300
2100
800
2000
500
300
900
2100
300
400
1500
800
2100
300
600
900
600
2100
400
2400
300
800
2300
500
800
300
2600
2300
500
300
400
400
2500
2600
2300
300
2500
300
300
2100
600
600
300
700
2400
700
300
400
300
400
700
800
800
2500
400
800
900
2400
2400
300
300
300
300
300
3000
300
600
2500
500
800
2300
2600
2300
600
2800
500
700
700
300
window:50  t1:5  t2:24  最大长度：300 
400
1500
400
1500
1600
500
700
1800
500
400
900
300
400
300
2200
500
300
600
500
400
800
800
400
300
700
300
300
2400
300
2100
800
2000
500
300
900
2100
300
400
1500
800
2100
300
600
900
600
2100
400
2400
300
800
2300
500
800
300
2600
2300
500
300
400
400
2500
2600
2300
300
2500
300
300
2100
600
600
300
700
2400
700
300
400
300
400
700
800
800
2500
400
800
900
2400
2400
300
300
300
300
300
3000
300
600
2500
500
800
2300
2600
2300
600
2800
500
700
700
300
window:50  t1:5  t2:24  最大长度：300 
400
1500
400
1500
1600
500
700
1800
500
400
900
300
400
30

900
300
400
300
2200
500
300
600
500
400
800
800
400
300
700
300
300
2400
300
2100
800
2000
500
300
900
2100
300
400
1500
800
2100
300
600
900
600
2100
400
2400
300
800
2300
500
800
300
2600
2300
500
300
400
400
2500
2600
2300
300
2500
300
300
2100
600
600
300
700
2400
700
300
400
300
400
700
800
800
2500
400
800
900
2400
2400
300
300
300
300
300
3000
300
600
2500
500
800
2300
2600
2300
600
2800
500
700
700
300
window:50  t1:5  t2:24  最大长度：300 
400
1500
400
1500
1600
500
700
1800
500
400
900
300
400
300
2200
500
300
600
500
400
800
800
400
300
700
300
300
2400
300
2100
800
2000
500
300
900
2100
300
400
1500
800
2100
300
600
900
600
2100
400
2400
300
800
2300
500
800
300
2600
2300
500
300
400
400
2500
2600
2300
300
2500
300
300
2100
600
600
300
700
2400
700
300
400
300
400
700
800
800
2500
400
800
900
2400
2400
300
300
300
300
300
3000
300
600
2500
500
800
2300
2600
2300
600
2800
500
700
700
300
window:50  t1:5  t2:24  最大长度：300 
400
1500
400
1500
1600
500
700
1800
500
400
900
300
400
30

KeyboardInterrupt: 

In [ ]:
'''3.切分数据'''
#输出最大长度
max_lenth = 0 
for i in range(len(end_point)):
    if(max_lenth < end_point[i]- start_point[i]):
        max_lenth = end_point[i]- start_point[i]
print("单个样本最大长度：%d" % max_lenth)
for i in range(len(end_point)):
    aaa = s_acc[:,start_point[i]:end_point[i]]
    print(aaa.shape)
